In [2]:
import sys
sys.version

'3.5.2 |Anaconda 4.2.0 (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]'

In [3]:
import csv as csv # csv 라는 모듈을 읽고 별칭은 csv로 사용함
import numpy as np

csv_file_object = csv.reader(open('./train.csv', 'rt'))
header = next(csv_file_object)

data=[]
for row in csv_file_object:
    data.append(row)
# convert from a list to an array
data = np.array(data)

# 2차원 배열의 value 접근방법
# [처음 시작 offset : 마지막 offset : incremental/decremental step], column number]
# 문자열은 str
number_passaenger=np.size(data[0::,0])
number_survived=np.sum(data[0::,1].astype(np.float))
prpportion_survivors = number_survived / number_passaenger

# 살아남은 사람의 비율 891명 중 38.3%가 살아남음
prpportion_survivors

0.38383838383838381

In [4]:
women_only_stats = data[0::,4] == "female"
men_only_stats = data[0::,4] != "female"

In [5]:
# true값을 1로 바꿔줌
women_onboard = data[women_only_stats, 1].astype(np.float)
men_onboard = data[men_only_stats, 1].astype(np.float)
# 그 후, 남자, 여자의 생존 비율을 계산 할 수 있음
proportion_women_survived = np.sum(women_onboard) / np.size(women_onboard)
proportion_men_survived = np.sum(men_onboard) / np.size(men_onboard)

print("proportion of women who survived is %s" % proportion_women_survived)
print("proportion of men who survived is %s" % proportion_men_survived)

proportion of women who survived is 0.742038216561
proportion of men who survived is 0.188908145581


In [6]:
test_file = open('./test.csv', 'rt')
test_file_object = csv.reader(test_file)
header = next(test_file_object)

In [7]:
prediction_file = open('genderbasedmodel.csv', 'wt')
prediction_file_object=csv.writer(prediction_file)

In [8]:
# It create simple classification model using test data sets.
# If passenger is femals, we predict that she is survived.
prediction_file_object.writerow(['PassengerId','Survived'])
for row in test_file_object:
    if row[3] == 'female':
        prediction_file_object.writerow([row[0],'1'])
    else:
        prediction_file_object.writerow([row[0],'0'])
test_file.close()
prediction_file.close()

In [16]:
fare_ceiling = 40
data[data[0::,9].astype(np.float) >= fare_ceiling, 9] = fare_ceiling - 1.0

fare_bracket_size = 10
number_of_price_brackets = fare_ceiling / fare_bracket_size

number_of_classes = 3
number_of_classes = len(np.unique(data[0::,2]))

# 3차원 배열 2*3*4를 생성함
survival_table = np.zeros((2, number_of_classes, number_of_price_brackets))

print(survival_table)

[[[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]]


C:\Users\jaegeun\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [25]:
# 3*4번 for문을 돌면서
for i in range(number_of_classes):
    for j in range(int(number_of_price_brackets)):
        # 여자와 남자 중 살아 남은 사람들 중에 승선 티켓의 종류는? 지불한 금액에 따라서 분류함
        # women_only_stats, men_only_stats 데이터를 생성함
        # 그 후, survival_table에 input함
        women_only_stats = data[                          #Which element           
                         (data[0::,4] == "female")    #is a female
                       &(data[0::,2].astype(np.float) #and was ith class
                             == i+1)                       
                       &(data[0::,9].astype(np.float)  #was greater 
                            >= j*fare_bracket_size)   #than this bin              
                       &(data[0::,9].astype(np.float)  #and less than
                            < (j+1)*fare_bracket_size)#the next bin    
                          , 1]                        #in the 2nd col 
  
        
        men_only_stats = data[                          #Which element           
                         (data[0::,4] == "male")    #is a female
                       &(data[0::,2].astype(np.float) #and was ith class
                             == i+1)                       
                       &(data[0::,9].astype(np.float)  #was greater 
                            >= j*fare_bracket_size)   #than this bin              
                       &(data[0::,9].astype(np.float)  #and less than
                            < (j+1)*fare_bracket_size)#the next bin    
                          , 1]
        
        survival_table[0,i,j] = np.mean(women_only_stats.astype(np.float))
        survival_table[1,i,j] = np.mean(men_only_stats.astype(np.float))
        
survival_table[ survival_table != survival_table] = 0.

print(survival_table)
# 테이블을 분석해 보면
# 여자 중, 1 class, 1 price_bracket인 사람 들 중 살아 남은 사람은 없는것을 의미함=0(mean:평균값)
# 여자 중, 2 class, 4 price_bracket인 사람 들은 모두 삻아 남은 것을 의미함=1

[[[ 0.          0.          0.83333333  0.97727273]
  [ 0.          0.91428571  0.9         1.        ]
  [ 0.59375     0.58139535  0.33333333  0.125     ]]

 [[ 0.          0.          0.4         0.38372093]
  [ 0.          0.15873016  0.16        0.21428571]
  [ 0.11153846  0.23684211  0.125       0.24      ]]]


C:\Users\jaegeun\Anaconda3\lib\site-packages\numpy\core\_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
C:\Users\jaegeun\Anaconda3\lib\site-packages\numpy\core\_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [26]:
# survival_table에 있는 값이 >= 0.5 면 살아남는다고 예측
# survival_table에 있는 값이 < 0.5면 죽었다고 예측
survival_table[survival_table < 0.5] = 0
survival_table[survival_table >= 0.5] = 1

print(survival_table)

[[[ 0.  0.  1.  1.]
  [ 0.  1.  1.  1.]
  [ 1.  1.  0.  0.]]

 [[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]]


In [27]:
test_file = open('./test.csv', 'rt')
test_file_object = csv.reader(test_file)
header = next(test_file_object)
prediction_file = open('./genderclassmodel.csv','wt')
p = csv.writer(prediction_file)
p.writerow(['PassengerId','Survived'])

for row in test_file_object:
    for j in range(int(number_of_price_brackets)):
        try:
            row[8] = float(row[8])
        except:
            bin_fare = 3 - float(row[1])
            break
        if row[8] > fare_ceiling:
            bin_fare = number_of_price_brackets-1
            break
        if row[8] >= j*fare_bracket_size and row[8] < (j+1) * fare_bracket_size:
            bin_fare = j
            break
    if row[3] == 'female':
        p.writerow([row[0], "%d" % int(survival_table[0, float(row[1])-1, bin_fare])])
    else:
        p.writerow([row[0], "%d" % int(survival_table[1, float(row[1])-1, bin_fare])])
test_file.close()
prediction_file.close()

C:\Users\jaegeun\Anaconda3\lib\site-packages\ipykernel\__main__.py:24: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\jaegeun\Anaconda3\lib\site-packages\ipykernel\__main__.py:22: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
